# Complex problems in Industry 

In industry environment, several parameters can increase the complexity of the scheduling problem :
- number of task (>= hundreds task to schedule)
- varying resource availability
- individual worker to be taken into account : different availability of workers, skills and qualifications different from one worker to another one.
- specific constraints of the industrial process :
    - synchronisation of task, generalized precedence constraints (with time lags).
    - possibility to pause and resume task (preemptive scheduling problems)
...

In this notebook you'll code a CP model for RCPSP problem that you discovered in the first notebook of this course.

<div class="alert alert-warning">
    <b>Exercise</b> appear in yellow.
</div>


<div class="alert alert-success">
    <b>Helps and hints</b> appear in green.
</div>

In [ ]:
# Usefull imports.
import sys, os
os.environ["DO_SKIP_MZN_CHECK"] = "1"
# patching asyncio so that applications using async functions can run in jupyter
import nest_asyncio
nest_asyncio.apply()
import logging
logging.basicConfig(level=logging.INFO)
import time
from pprint import pprint
from discrete_optimization.rcpsp.rcpsp_model import RCPSPModel, RCPSPSolution
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view, plot_task_gantt

## Loading an RCPSP instance from industrial use case
Some files are in the data/ folder of this repository and can be parsed into a ```RCPSPModel``` object.

In [ ]:
from script_utils.json_format import load_any_dict, load_any_json, load_instance_msrcpsp
this_folder = os.path.dirname(os.getcwd())
print(this_folder)
folder_files_example = os.path.join(this_folder, "scheduling_newcourse/data/advanced/")
list_files = os.listdir(folder_files_example)
name_instance = "instance_index_1_multiskill_False_calendar_False_specconstraints_False_preemptive_False.json"
rcpsp_model: RCPSPModel = load_any_json(os.path.join(folder_files_example, name_instance))
print(rcpsp_model)

## Computing dummy solution
As simpler scheduling problem, we can still compute basic solution and plot it.

In [ ]:
dummy_solution = rcpsp_model.get_dummy_solution()
print(rcpsp_model.evaluate(dummy_solution))
print(rcpsp_model.evaluate(dummy_solution)["makespan"])

In [ ]:
plot_ressource_view(rcpsp_model, dummy_solution)
plot_task_gantt(rcpsp_model, dummy_solution)

# Code a CP model for RCPSP : 

<div class="alert alert-warning">
<b>Problem #1: (medium/hard)</b><br />

Coding a CP model for RCPSP
</div>

<div class="alert alert-success">
Don't hesitate to have a look at the slide deck defining precisely the constraints of the RCPSP problem, along with notebook n°1.
</div>

### Helpers function
We provide some function to return important date from the RCPSP model that we instanciated above.

In [ ]:
from typing import List, Hashable, Dict
def get_resource_names(rcpsp_model: RCPSPModel)->List[str]:
    return rcpsp_model.resources_list

def get_resource_capacity(rcpsp_model: RCPSPModel, res_name: str)->int:
    return rcpsp_model.get_max_resource_capacity(res_name)

def get_all_resource_capacity(rcpsp_model: RCPSPModel)->Dict[str, int]:
    return {res: get_resource_capacity(rcpsp_model, res) for res in get_resource_names(rcpsp_model)}

def get_tasks_list(rcpsp_model: RCPSPModel)->List[Hashable]:
    return rcpsp_model.tasks_list

def get_successors_of_task(rcpsp_model: RCPSPModel, task_name: Hashable)->List[Hashable]:
    return rcpsp_model.successors.get(task_name, [])

def get_all_successors(rcpsp_model: RCPSPModel)->Dict[Hashable, List[Hashable]]:
    return rcpsp_model.successors

def get_resource_demand(rcpsp_model: RCPSPModel, task_name: Hashable, res_name: str)->int:
    return rcpsp_model.mode_details[task_name][1].get(res_name, 0)

def get_duration(rcpsp_model: RCPSPModel, task_name: Hashable)->int:
    return rcpsp_model.mode_details[task_name][1]["duration"]

def get_duration_map(rcpsp_model: RCPSPModel)->Dict[Hashable, int]:
    return {t: get_duration(rcpsp_model, t) for t in get_tasks_list(rcpsp_model)}

def get_rcpsp_solution(rcpsp_model: RCPSPModel, starts_dict: Dict[Hashable, int],
                       ends_dict: Dict[Hashable, int])->RCPSPSolution:
    return RCPSPSolution(problem=rcpsp_model,
                         rcpsp_schedule={t: {"start_time": starts_dict[t], "end_time": ends_dict[t]} 
                                         for t in starts_dict},
                         rcpsp_modes=[1]*rcpsp_model.n_jobs_non_dummy, rcpsp_schedule_feasible=True)

In [ ]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

<div class="alert alert-warning">
First part : create the needed variables. Please add in comment what the variables represent.
</div>

<div class="alert alert-success">
Should be quite close to job shop variables. 
You can use the dummy solution to get upper bounds of the makespan.
</div>

In [ ]:
# TO FILL : add new variable to the model.

In [ ]:
# %load correction/nb3_handson_1.py

<div class="alert alert-warning">
First part : create the Constraints, commenting what they represent.
</div>

<div class="alert alert-success">
Look at the slide deck of this morning, along with the first notebook.
</div>

In [ ]:
# To fill

In [ ]:
# %load correction/nb3_handson_2.py

<div class="alert alert-warning">
Third part : create the objective function variable, and set minimization to the model
</div>

In [ ]:
# TO FILL

In [ ]:
# %load correction/nb3_handson_3.py

### Solving the model

To observe the progress of the solver, you can put a solution callback to print intermediary solutions.

In [ ]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions quality"""

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._solution_count = 0

    def on_solution_callback(self):
        print("Solution count : ", self._solution_count,
              "Objective : ", self.ObjectiveValue())
        self._solution_count += 1

    def solution_count(self):
        return self._solution_count

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30
status = solver.Solve(model, SolutionPrinter())
status_human = solver.StatusName(status)

In [ ]:
solver.ObjectiveValue()

<div class="alert alert-warning">
Last part : Retrieve the solution in a dictionnary : starts={key(task_name): start_time} and ends={key(task_name): end_time}
</div>

In [ ]:
# TO FILL
starts_value = {t: None for t in tasks_list}
ends_value = {t: None for t in tasks_list}
# RETRIEVE THE VALUES FROM THE SOLVER RESULTS example if you have a variable starts[t] for a given task,
# you can get its value by : solver.Value(starts[t])

In [ ]:
# %load correction/nb3_handson_4.py

#### Create an object RCPSPSolution from the output of the CP Solver and plot it.

In [ ]:
solution = get_rcpsp_solution(rcpsp_model=rcpsp_model, starts_dict=starts_value,
                              ends_dict=ends_value)
satisfy = rcpsp_model.satisfy(solution)
evaluation = rcpsp_model.evaluate(solution)
print("Solution satisfy the constraints ", satisfy)
print(evaluation)

In [ ]:
plot_ressource_view(rcpsp_model, solution)
plot_task_gantt(rcpsp_model, solution)

#### Export the results 

In [ ]:
import json
import pickle
import datetime
folder_export = os.path.join(this_folder, "export_result/")
if not os.path.exists(folder_export):
    os.makedirs(folder_export)
def export(rcpsp_solution: RCPSPSolution, name_instance: str, name_file:str, method:str="cp"):
    d = {"schedule": {t: {"start_time": int(rcpsp_solution.rcpsp_schedule[t]["start_time"]),
                          "end_time": int(rcpsp_solution.rcpsp_schedule[t]["end_time"])}
                      for t in rcpsp_solution.rcpsp_schedule},
         "makespan": int(rcpsp_solution.problem.evaluate(rcpsp_solution)["makespan"]),
         "satisfy": rcpsp_solution.problem.satisfy(rcpsp_solution),
         "method": method,
         "name_instance": name_instance}
    path_export = os.path.join(folder_export, name_file)
    json.dump(d, open(path_export, 'w'), indent=4)
    print("solution exported in "+path_export)
    return path_export

now = datetime.datetime.now()    
path_export = export(solution, 
                     name_instance=name_instance, 
                     name_file="my_results_"+str(now)+".json",
                     method="cp")

## More complex instance

<div class="alert alert-danger">
    <b>Optional Exercise</b>
    This is a bonus part of this handson. Go there if you well understood the previous part and got a feasible solution to the classical RCPSP problem
</div>

In [ ]:
name_instance_sc = "instance_index_0_multiskill_False_calendar_False_specconstraints_True_preemptive_False.json"
rcpsp_model_sc: RCPSPModel = load_any_json(os.path.join(folder_files_example, name_instance_sc))
print(rcpsp_model_sc)

### Additional constraints description

#### Starting and ending time window 
in the 2 next cell, you have constraints on starting time and ending time for most of the tasks.
Let $ws_{t,0},ws_{t,1} = startwindow[t]$ then the constraints is : 

- if $ws_{t,0} != None$ : $start_t>=ws_{t,0}$
- if $ws_{t,1} != None$ : $start_t<=ws_{t,1}$

Similarly : $we_{t,0},we_{t,1} = endwindow[t]$

- if $we_{t,0} != None$ : $end_t>=we_{t,0}$
- if $we_{t,1} != None$ : $end_t<=we_{t,1}$


In [ ]:
rcpsp_model_sc.special_constraints.start_times_window

In [ ]:
rcpsp_model_sc.special_constraints.end_times_window

#### Synchronisation constraints

1)  $\forall (i,j)\in {start\_together}, start_i==start_j$
2)  $\forall (i,j)\in {start\_at\_end}, start_j=end_i$
3)  $\forall (i,j,{offset}) \in {start\_after\_nunit}, start_j \geq start_i+offset$
4)  $\forall (i,j,{offset}) \in {start\_at\_end\_plus\_offset}, start_j\geq end_i+\text{offset}$

In [ ]:
rcpsp_model_sc.special_constraints.start_together # List of (i,j)
rcpsp_model_sc.special_constraints.start_at_end # List of (i,j)
rcpsp_model_sc.special_constraints.start_after_nunit # List of (i,j,offset)
rcpsp_model_sc.special_constraints.start_at_end_plus_offset # List of (i,j,offset)

### CP modeling with additional constraints

<div class="alert alert-danger">
    <b>Optional Exercise</b>
    code the cp model !
</div>

In [ ]:
# CP MODEL 
model = cp_model.CpModel()

In [ ]:
# %load correction/nb3_handson_bonus.py

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30
status = solver.Solve(model, SolutionPrinter())
status_human = solver.StatusName(status)
print(status_human)
starts_value = {t: solver.Value(starts[t]) for t in tasks_list}
ends_value = {t: solver.Value(ends[t]) for t in tasks_list}

In [ ]:
logging.basicConfig(level=logging.DEBUG)
solution = get_rcpsp_solution(rcpsp_model=rcpsp_model_sc, 
                              starts_dict=starts_value,
                              ends_dict=ends_value)
satisfy = rcpsp_model_sc.satisfy(solution)
evaluation = rcpsp_model_sc.evaluate(solution)
print("Solution satisfy the constraints ", satisfy)
print(evaluation)
plot_ressource_view(rcpsp_model_sc, solution)
plot_task_gantt(rcpsp_model_sc, solution)

## Take into account resource availability in the problem

In real life, the resource and worker are not available through the all horizon. Therefore, the resource capacity is not defined as a constant but as a vector giving $\forall t\in [0, horizon], k\in Resources, b_{k,t}\in \mathcal{N}$.
Let's build a rcpsp model with variable resource : 


In [ ]:
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file
filepath = [f for f in get_data_available() if "j1201_1.sm" in f][0]
rcpsp_model = parse_file(filepath)

In [ ]:
from typing import Optional
import numpy as np
import random
def create_variable_resource(rcpsp_model: RCPSPModel,
                             new_horizon: Optional[int],
                             nb_breaks: int = 20):
    resource_availability = {r: np.full(new_horizon,
                                        rcpsp_model.get_max_resource_capacity(r))
                             for r in rcpsp_model.resources_list}
    for r in resource_availability:
        for j in range(nb_breaks):
            t = random.randint(0, len(resource_availability[r])-1)
            len_break = random.randint(1, 5)
            resource_availability[r][t:t+len_break] = 0
    return RCPSPModel(resources=resource_availability,
                      non_renewable_resources=rcpsp_model.non_renewable_resources,
                      mode_details=rcpsp_model.mode_details,
                      successors=rcpsp_model.successors,
                      horizon=new_horizon-1,
                      tasks_list=rcpsp_model.tasks_list,
                      source_task=rcpsp_model.source_task,
                      sink_task=rcpsp_model.sink_task)

In [ ]:
rcpsp_model = create_variable_resource(rcpsp_model, new_horizon=rcpsp_model.horizon, nb_breaks=10)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1)
for r in rcpsp_model.resources_list:
    ax.plot(rcpsp_model.get_resource_availability_array(r), label=r)
ax.legend()

<div class="alert alert-danger">
    <b>Optional Exercise</b>
    code the cp model of the RCPSP with variable resource availability
</div>

Please write here the ideas you have, if any. You don't have to be fast enough to code everything !

"



"

In [ ]:
# %load correction/nb3_handson_bonus_variable_resource.py

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30
status = solver.Solve(model, SolutionPrinter())
status_human = solver.StatusName(status)
print(status_human)
starts_value = {t: solver.Value(starts[t]) for t in tasks_list}
ends_value = {t: solver.Value(ends[t]) for t in tasks_list}
logging.basicConfig(level=logging.DEBUG)

In [ ]:
solution = get_rcpsp_solution(rcpsp_model=rcpsp_model, 
                              starts_dict=starts_value,
                              ends_dict=ends_value)
satisfy = rcpsp_model.satisfy(solution)
evaluation = rcpsp_model.evaluate(solution)
print("Solution satisfy the constraints ", satisfy)
print(evaluation)
plot_ressource_view(rcpsp_model, solution)
plot_task_gantt(rcpsp_model, solution)